# Association Ruling Mining
## Objective

This notebook focuses on rule generation with the different data processing of the other notebooks.

## Data Loading and Preprocessing
- **Data Sources**: We utilized CSV files from previous notebooks
- **Preprocessing Steps**:
  - Loading dta from csv files and removing any unnecessary and unused columns.
  - Assign meaningful column names based on the data structure.
  - Convert the 'Date' column to datetime format for better manipulation.
  - Merge databases and provide meaningful attribute names.
  - Create meaningful one hot encoded data attributes for rule generation.
  - Drop attributes that are not one hot encoded.

## Analysis Overview
- **Rule Generation**:
  - Generated a frequent itemsets with a support threshold of 5%.
  - Generated association rulles with a support threshold of 5% and a confidence threshold of 80%


## Results
- **Rules**:
- antecedents(Yearly Inflation Increase, COVID-19 Pandemic) ->	consequents(Democrat Term)
- antecedents(Democrat Term, COVID-19 Pandemic) ->	consequents(Yearly Inflation Increase)
- antecedents(Gas Price Increase, Yearly Inflation Increase, Democrat Term) ->	consequents(Oil Direction)



## Conclusion
- Gas Price Increase, Yearly Iflation Increase, and Demcrat term usually results in increase of oil prices.






In [42]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("Data/Data1_Regular_Conventional.csv", header=2)
df = df.iloc[:-1, :2]
new_headers = ['Date', 'Gas Price']  # Specify your new column names here
df['Date'] = pd.to_datetime(df['Date'])
df.columns = new_headers
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year


df = df.set_index('Date').resample('M').first().reset_index()
df

,Date,Gas Price,Month,Year
0,1990-08-31,1.191,8,1990
1,1990-09-30,1.242,9,1990
2,1990-10-31,1.321,10,1990
3,1990-11-30,1.334,11,1990
4,1990-12-31,1.341,12,1990
...,...,...,...,...
400,2023-12-31,3.104,12,2023
401,2024-01-31,2.966,1,2024
402,2024-02-29,3.021,2,2024
403,2024-03-31,3.243,3,2024


In [43]:
df['Price Change'] = df['Gas Price'].diff()

# Define a function to label price changes as 'Up', 'Down', or 'No Change'
def label_change(change):
    if change > 0:
        return 1
    elif change < 0:
        return 0
    else:
        return 0

# Apply the function to create the 'Change Direction' column
df['Gas Price Increase'] = df['Price Change'].apply(label_change)


# Display the DataFrame with the new columns
df

,Date,Gas Price,Month,Year,Price Change,Gas Price Increase
0,1990-08-31,1.191,8,1990,NaN,0
1,1990-09-30,1.242,9,1990,0.051,1
2,1990-10-31,1.321,10,1990,0.079,1
3,1990-11-30,1.334,11,1990,0.013,1
4,1990-12-31,1.341,12,1990,0.007,1
...,...,...,...,...,...,...
400,2023-12-31,3.104,12,2023,-0.141,0
401,2024-01-31,2.966,1,2024,-0.138,0
402,2024-02-29,3.021,2,2024,0.055,1
403,2024-03-31,3.243,3,2024,0.222,1


In [52]:
# adding inflation

new_headers = ['Date', 'Inflation Rate']

df_inflation = pd.read_csv("Data/Inflation_rate_in_US_yearly.csv")
df_inflation.columns = new_headers
df_inflation['Date'] = pd.to_datetime(df_inflation['Date'])
df_inflation['Year'] = df_inflation['Date'].dt.year
df_inflation.drop(columns=['Date'], inplace=True)

df_inflation['Yearly Inflation Change'] = df_inflation['Inflation Rate'].diff()
df_inflation['Yearly Inflation Increase'] = df_inflation['Yearly Inflation Change'].apply(label_change)


merged_df = pd.merge(df, df_inflation, on='Year', how='inner')
merged_df.reset_index()




# adding world events

iraq_war_start, iraq_war_end = '2003-01-01', '2003-12-31'
financial_crisis_start, financial_crisis_end = '2007-12-01', '2009-06-30'
covid_start, covid_end = '2020-01-01', '2023-12-31'  # Adjust end date as per your data availability

# Create new columns for world events
merged_df['Iraq War'] = (merged_df['Date'] >= iraq_war_start) & (merged_df['Date'] <= iraq_war_end)
merged_df['Financial Crisis'] = (merged_df['Date'] >= financial_crisis_start) & (merged_df['Date'] <= financial_crisis_end)
merged_df['COVID-19 Pandemic'] = (merged_df['Date'] >= covid_start) & (merged_df['Date'] <= covid_end)

# Convert boolean values to integers (0 or 1)
merged_df[['Iraq War', 'Financial Crisis', 'COVID-19 Pandemic']] = merged_df[['Iraq War', 'Financial Crisis', 'COVID-19 Pandemic']].astype(int)




# adding presidential terms

presidential_terms = {
    'Democrat': [(1993, 1996), (1997, 2000), (2009, 2012), (2013, 2016), (2021, 2024)],
    'Republican': [(1989, 1992), (2001, 2004), (2005, 2008), (2017, 2020)]
}

# Create new columns for presidential terms
merged_df['Democrat Term'] = 0
merged_df['Republican Term'] = 0

# Iterate through the presidential terms dictionary
for party, terms in presidential_terms.items():
    for term_start, term_end in terms:
        merged_df.loc[(merged_df['Year'] >= term_start) & (merged_df['Year'] <= term_end), f'{party} Term'] = 1


# Oil prices
crude_oil_prices_path = "Data\Macrotrends-crude-oil-prices-daily.csv"
crude_oil_prices_data = pd.read_csv(crude_oil_prices_path, skiprows=9)
crude_oil_prices_data['Date'] = pd.to_datetime(crude_oil_prices_data['Date'])
crude_oil_prices_data = crude_oil_prices_data.set_index('Date').resample('M').first().reset_index()
crude_oil_prices_data['Oil Change'] = crude_oil_prices_data['Closing Value'].diff()
crude_oil_prices_data['Oil Direction'] = crude_oil_prices_data['Oil Change'].apply(label_change)




merged_df = pd.merge(merged_df, crude_oil_prices_data, on='Date', how='inner')
merged_df.reset_index()


merged_df = merged_df.drop(columns=['Date', 'Gas Price', 'Month', 'Year', 'Price Change', 'Inflation Rate', 'Yearly Inflation Change', 'Closing Value', 'Oil Change'])

merged_df



<>:56: SyntaxWarning: invalid escape sequence '\M'
<>:56: SyntaxWarning: invalid escape sequence '\M'
C:\Users\jobea\AppData\Local\Temp\ipykernel_17188\1874738911.py:56: SyntaxWarning: invalid escape sequence '\M'
  crude_oil_prices_path = "Data\Macrotrends-crude-oil-prices-daily.csv"


,Gas Price Increase,Yearly Inflation Increase,Iraq War,Financial Crisis,COVID-19 Pandemic,Democrat Term,Republican Term,Oil Direction
0,0,1,0,0,0,0,1,1
1,1,1,0,0,0,0,1,1
2,1,1,0,0,0,0,1,1
3,1,1,0,0,0,0,1,0
4,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...
384,0,1,0,0,1,1,0,0
385,0,1,0,0,1,1,0,0
386,0,1,0,0,1,1,0,0
387,1,1,0,0,1,1,0,1


In [53]:
from mlxtend.frequent_patterns import apriori, association_rules

frq_items = apriori(merged_df, min_support = 0.05, max_len=None, use_colnames = True)
frq_items.sort_values("support",ascending =False)


c:\Users\jobea\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
3,0.555270,(Democrat Term)
5,0.537275,(Oil Direction)
0,0.516710,(Gas Price Increase)
1,0.506427,(Yearly Inflation Increase)
4,0.444730,(Republican Term)
10,0.383033,"(Gas Price Increase, Oil Direction)"
17,0.311054,"(Oil Direction, Democrat Term)"
12,0.308483,"(Yearly Inflation Increase, Democrat Term)"
14,0.293059,"(Yearly Inflation Increase, Oil Direction)"
6,0.282776,"(Gas Price Increase, Yearly Inflation Increase)"


In [68]:
rules = association_rules(frq_items, metric ="confidence", min_threshold = 0.8)
rules = rules.sort_values('confidence', ascending =False)
rules.head(10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"(Yearly Inflation Increase, COVID-19 Pandemic)",(Democrat Term),0.061697,0.555270,0.061697,1.000000,1.800926,0.027438,inf,0.473973
1,"(Democrat Term, COVID-19 Pandemic)",(Yearly Inflation Increase),0.061697,0.506427,0.061697,1.000000,1.974619,0.030452,inf,0.526027
2,"(Gas Price Increase, Yearly Inflation Increase...",(Oil Direction),0.169666,0.537275,0.143959,0.848485,1.579237,0.052802,3.053985,0.441729


In [55]:
rules.describe()

c:\Users\jobea\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\jobea\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
c:\Users\jobea\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:4658: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t >= 0.5)


,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
count,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
mean,0.097686,0.532991,0.089117,0.949495,1.784928,0.036897,inf,0.480576
std,0.062336,0.024702,0.047494,0.087477,0.198176,0.013856,NaN,0.042535
min,0.061697,0.506427,0.061697,0.848485,1.579237,0.027438,3.053985,0.441729
25%,0.061697,0.521851,0.061697,0.924242,1.690082,0.028945,NaN,0.457851
50%,0.061697,0.537275,0.061697,1.000000,1.800926,0.030452,NaN,0.473973
75%,0.115681,0.546272,0.102828,1.000000,1.887773,0.041627,NaN,0.500000
max,0.169666,0.555270,0.143959,1.000000,1.974619,0.052802,inf,0.526027
